In [2]:
# this example combines models from several libraries

# got weird errors after XGB and LGBM models, but seems to run correctly. Errors
# mention changes in those modules

#imports
import os
import sys

from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

from sklearn.model_selection import train_test_split
from sklearn.datasets import load_breast_cancer

from combo.models.classifier_stacking import Stacking
from combo.utils.data import evaluate_print

import warnings

warnings.filterwarnings("ignore")

In [3]:
if __name__ == "__main__":
    # Define data file and read X and y
    random_state = 42
    X, y = load_breast_cancer(return_X_y=True)

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.4, random_state=random_state)

    # initialize a group of clfs from scikit-learn, xgboost, and LightGBM
    classifiers = [DecisionTreeClassifier(random_state=random_state),
                   LogisticRegression(random_state=random_state),
                   KNeighborsClassifier(),
                   RandomForestClassifier(random_state=random_state),
                   GradientBoostingClassifier(random_state=random_state),
                   LGBMClassifier(random_state=random_state),
                   XGBClassifier(random_state=random_state)]
    #make abbreviations for printing
    clf_names = ['DT', 'LR', 'KNN', 'RF', 'GBDT', 'LGB', 'XGB']

    # evaluate individual classifiers
    for i, clf in enumerate(classifiers):
        #train data per classifier
        clf.fit(X_train, y_train)
        #predict per
        y_test_predict = clf.predict(X_test)
        #print per
        evaluate_print(clf_names[i] + '   |   ', y_test, y_test_predict)

    print()
    
    #build stacking models
    #takes in list of classifiers, number of splits, does not shuffle data, keep original features for training does not use proba
    clf = Stacking(classifiers, n_folds = 4, shuffle_data = False, keep_original = True, use_proba = False, random_state = random_state)
    #train, predict, predict probability, print
    clf.fit(X_train, y_train)
    y_test_predict = clf.predict(X_test)
    y_test_predict_proba = clf.predict_proba(X_test)
    evaluate_print('Stacking | ', y_test, y_test_predict)

DT   |    Accuracy:0.9386, ROC:0.9383, F1:0.9521
LR   |    Accuracy:0.9649, ROC:0.9586, F1:0.9732
KNN   |    Accuracy:0.9561, ROC:0.9519, F1:0.9662
RF   |    Accuracy:0.9781, ROC:0.9716, F1:0.9833
GBDT   |    Accuracy:0.9605, ROC:0.9524, F1:0.9699
LGB   |    Accuracy:0.943, ROC:0.9417, F1:0.9556
[01:54:42] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
XGB   |    Accuracy:0.9737, ROC:0.9654, F1:0.98

[01:54:44] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[01:54:44] WARNING: C:/Users/Admin